Tutorial and code followed from here  -   https://mc.ai/neural-machine-translation-using-word-level-seq2seq-model/
dataset can be downloaded from here -- https://github.com/devm2024/nmt_keras/blob/master/fra.txt

In [14]:
from google.colab import files
uploaded = files.upload()

Saving fra.txt to fra.txt


In [0]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.model_selection import train_test_split
import copy
# Building a english to french translator

In [16]:
#changes the width of cells

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [0]:
df= pd.read_csv('fra.txt',sep = '\t',names=['eng','fr'])


In [0]:
dfcopy = copy.copy(df)
#taking only 1000 as I just need to understand how it's working , for actual implementation use complete dataset

In [19]:
df.shape
#eng and fr columns
df.head()
len(df)

149861

In [0]:
df.head()
df.iloc[11705]
df = df[:20000]

In [21]:
df.sample()


,eng,fr
15766,You're all alone.,Vous êtes tout seul.


In [22]:
df.head()



,eng,fr
0,Go.,Va !
1,Run!,Cours !
2,Run!,Courez !
3,Wow!,Ça alors !
4,Fire!,Au feu !


In [0]:
df.eng=df.eng.apply(lambda x: x.lower())
df.fr=df.fr.apply(lambda x: x.lower())

In [24]:
df.head()

,eng,fr
0,go.,va !
1,run!,cours !
2,run!,courez !
3,wow!,ça alors !
4,fire!,au feu !


In [0]:

# Take the length as 50
df.eng=df.eng.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
df.fr=df.fr.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

In [0]:
#punctuations
exclude = set(string.punctuation)
df.eng=df.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df.fr=df.fr.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
remove_digits = str.maketrans('', '', digits)
df.eng=df.eng.apply(lambda x: x.translate(remove_digits))
df.fr=df.fr.apply(lambda x: x.translate(remove_digits))

In [28]:
'''
Syntax : maketrans(str1, str2, str3)

Parameters :
str1 : Specifies the list of characters that need to be replaced.
str2 : Specifies the list of characters with which the characters need to be replaced.
str3 : Specifies the list of characters that needs to be deleted.

Returns : Returns the translation table which specifies the conversions that can be used by translate()
'''

'\nSyntax : maketrans(str1, str2, str3)\n\nParameters :\nstr1 : Specifies the list of characters that need to be replaced.\nstr2 : Specifies the list of characters with which the characters need to be replaced.\nstr3 : Specifies the list of characters that needs to be deleted.\n\nReturns : Returns the translation table which specifies the conversions that can be used by translate()\n'

In [29]:
df.fr = df.fr.apply(lambda x : 'START_ '+ x + ' _END')
df.head()

,eng,fr
0,go,START_ va _END
1,run,START_ cours _END
2,run,START_ courez _END
3,wow,START_ ça alors _END
4,fire,START_ au feu _END


### post filteration part

In [0]:
all_eng_words=set()
for eng in df.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
    
all_french_words=set()
for fr in df.fr:
    for word in fr.split():
        if word not in all_french_words:
            all_french_words.add(word)

In [31]:
print(len(all_eng_words))
print(len(all_french_words))

3445
7196


In [32]:
fr_lenght_list=[]
for l in df.fr:
    fr_lenght_list.append(len(l.split(' ')))

eng_lenght_list=[]
for l in df.eng:
    eng_lenght_list.append(len(l.split(' ')))
print(np.max(fr_lenght_list))
print(np.max(eng_lenght_list))
max_fr = np.max(fr_lenght_list)
max_eng = np.max(eng_lenght_list)

14
5


In [0]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_french_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_french_words)
# del all_eng_words, all_french_words

In [0]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [35]:
len(df.fr)*58*num_decoder_tokens


8347360000

encoder_input_data is a matrix of dimentions = no_of_sentences*max_sentence_length  


decoder_input_data is a matrix of dimentions = no_of_sentences*max_sentence_length  

decoder_target_data is a matrix of dimensions = no_of_sentences*max_sentence_length*no_of_distinct_words

In [36]:
print(len(df.fr),num_decoder_tokens)

20000 7196


In [0]:

encoder_input_data = np.zeros(
    (len(df.eng),max_eng ),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(df.fr), max_fr),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(df.fr), max_fr, num_decoder_tokens),
    dtype='float32')

In [0]:

for i, (input_text, target_text) in enumerate(zip(df.eng, df.fr)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

## The model making part


In [0]:
embedding_size = 50
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

<b>Encoder Model</b>


In [41]:
encoder_inputs = Input(shape = (None,))
encoder_embeddings = Embedding(num_encoder_tokens,embedding_size)(encoder_inputs)
E_LSTM = LSTM(50,return_state=True)
encoder_output , hidden_state , memory_cell = E_LSTM(encoder_embeddings)
encoder_states = [hidden_state,memory_cell]



Instructions for updating:
Colocations handled automatically by placer.


<b> Decoder Model </b>

In [0]:
decoder_inputs = Input(shape = (None,))
decoder_embeddings = Embedding(num_decoder_tokens,embedding_size)
final_dex = decoder_embeddings(decoder_inputs)
decoder_LSTM = LSTM(50,return_sequences=True,return_state=True)
decoder_outputs ,_ , _ = decoder_LSTM(final_dex, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens,activation='softmax') # total no of unique words
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc']) 

In [43]:
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     172250      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     359800      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [44]:

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=20,
          validation_split=0.05)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 19000 samples, validate on 1000 samples
Epoch 1/20
19000/19000 [==============================] - 16s 840us/step - loss: 2.0014 - acc: 0.0709 - val_loss: 2.1141 - val_acc: 0.0714
Epoch 2/20
19000/19000 [==============================] - 12s 634us/step - loss: 1.7118 - acc: 0.0786 - val_loss: 2.0013 - val_acc: 0.0714
Epoch 3/20
19000/19000 [==============================] - 12s 645us/step - loss: 1.6041 - acc: 0.0884 - val_loss: 1.9226 - val_acc: 0.0764
Epoch 4/20
19000/19000 [==============================] - 12s 637us/step - loss: 1.5261 - acc: 0.0972 - val_loss: 1.8519 - val_acc: 0.0856
Epoch 5/20
19000/19000 [==============================] - 12s 640us/step - loss: 1.4682 - acc: 0.1026 - val_loss: 1.7951 - val_acc: 0.0923
Epoch 6/20
19000/19000 [==============================] - 12s 632us/step - loss: 1.4168 - acc: 0.1079 - val_loss: 1.7374 - val_acc

In [45]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 50)          172250    
_________________________________________________________________
lstm_1 (LSTM)                [(None, 50), (None, 50),  20200     
Total params: 192,450
Trainable params: 192,450
Non-trainable params: 0
_________________________________________________________________


In [0]:
 decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2=decoder_embeddings(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_LSTM(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [0]:

def decode_sequence(input_seq):
    #input_seq is  [[1607.   80. 3062.    0.    0.]] for 14077
    print("input is ",input_seq)
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    print("state_value is ", states_value)
    '''
    state_value is  [array([[-0.7356597 , -0.9993317 , -0.8581535 ,  0.9991428 ,  0.86860234,
        -0.88219815,  0.6262745 , -0.9992849 ,  0.9986833 ,  0.99975616,
        -0.96406317, -0.8911646 , -0.93668246,  0.8071924 , -0.9070601 ,
        -0.9983337 , -0.6851529 , -0.5785509 ,  0.9981099 , -0.45333877,
        -0.9723586 , -0.92026615, -0.5631677 , -0.9855832 ,  0.53639436,
         0.7567345 ,  0.8314614 ,  0.98461056, -0.02521831, -0.90126914,
         0.9989688 ,  0.8336466 , -0.9887398 , -0.7309139 , -0.52631956,
        -0.6803367 , -0.9744914 , -0.9639372 ,  0.8591712 , -0.07826253,
         0.9617582 , -0.7269689 , -0.9957466 , -0.6216118 , -0.9720878 ,
         0.89779437,  0.93081397, -0.9966131 ,  0.9854572 ,  0.9793006 ]],
      dtype=float32), array([[-3.2502072 , -4.0018077 , -2.370447  ,  3.8773093 ,  3.6909149 ,
        -3.2669096 ,  3.1789834 , -3.9679527 ,  3.662548  ,  4.506039  ,
        -2.5889246 , -1.4275558 , -2.4326057 ,  1.1189184 , -3.5042324 ,
        -3.5447226 , -3.0464299 , -2.1128592 ,  3.4816515 , -2.5415885 ,
        -2.1338346 , -1.5907625 , -0.63746005, -2.462637  ,  4.776198  ,
         3.2632422 ,  3.4211106 ,  2.429748  , -0.02522366, -1.4789395 ,
         3.7848165 ,  1.8303449 , -2.5869899 , -0.9306867 , -0.8429435 ,
        -2.9611928 , -2.174526  , -1.9987224 ,  1.2901706 , -0.0784229 ,
         1.9688332 , -3.3909166 , -3.0755305 , -2.5341778 , -2.1288924 ,
         1.4607303 ,  1.664449  , -3.1896398 ,  2.4582543 ,  2.2801971 ]],
      dtype=float32)]
      
     ''' 
    
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1)) 
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']
    print("target_seq is ",target_seq[0,0] )
    
    #target_seq is  1.0

    
    
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
      output_tokens, h, c = decoder_model.predict(
          [target_seq] + states_value)
      print("output_tokens is ", output_tokens)

      # Sample a token
      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      print("sampled_token_index is ",sampled_token_index)
      sampled_char = reverse_target_char_index[sampled_token_index]
      decoded_sentence += ' '+sampled_char

      print("decoded sentence is ",decoded_sentence)
      # Exit condition: either hit max length
      # or find stop character.
      if sampled_char == '_END' or len(decoded_sentence) > 52 :
        stop_condition = True

      # Update the target sequence (of length 1).
      target_seq = np.zeros((1,1))
      target_seq[0, 0] = sampled_token_index

      # Update states
      states_value = [h, c]
      
      '''
    output_tokens is  [[[1.9580897e-04 1.4204055e-10 3.0356929e-05 ... 3.8483760e-12
       1.3986213e-14 2.2031972e-09]]]
    sampled_token_index is  929
    decoded sentence is   cest
    output_tokens is  [[[4.6192162e-04 9.7681152e-07 1.8441869e-04 ... 1.9450983e-06
       1.9787763e-06 3.8920575e-06]]]
    sampled_token_index is  6753
    decoded sentence is   cest un
    output_tokens is  [[[4.5321140e-05 9.9775898e-06 2.7055165e-04 ... 3.3116699e-04
       7.6128467e-04 3.4273871e-05]]]
    sampled_token_index is  4784
    decoded sentence is   cest un peu
    output_tokens is  [[[2.2299837e-03 3.8076034e-06 2.4568696e-02 ... 7.3837878e-06
       6.9453758e-06 2.5249241e-05]]]
    sampled_token_index is  1566
    decoded sentence is   cest un peu de
    output_tokens is  [[[7.1985596e-05 3.0220974e-06 2.2143837e-04 ... 2.4737383e-05
       1.8406533e-04 3.3893932e-05]]]
    sampled_token_index is  1131
    decoded sentence is   cest un peu de colère
    output_tokens is  [[[4.6538683e-03 9.3184195e-07 7.0089167e-01 ... 5.0056883e-07
        2.9316115e-07 1.3825561e-05]]]
    sampled_token_index is  2
    decoded sentence is   cest un peu de colère _END
    -
    
    
    '''
      
    return decoded_sentence

In [66]:
for seq_index in [14077]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', df.eng[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

input is  [[1607.   80. 3062.    0.    0.]]
state_value is  [array([[-0.7356597 , -0.9993317 , -0.8581535 ,  0.9991428 ,  0.86860234,
        -0.88219815,  0.6262745 , -0.9992849 ,  0.9986833 ,  0.99975616,
        -0.96406317, -0.8911646 , -0.93668246,  0.8071924 , -0.9070601 ,
        -0.9983337 , -0.6851529 , -0.5785509 ,  0.9981099 , -0.45333877,
        -0.9723586 , -0.92026615, -0.5631677 , -0.9855832 ,  0.53639436,
         0.7567345 ,  0.8314614 ,  0.98461056, -0.02521831, -0.90126914,
         0.9989688 ,  0.8336466 , -0.9887398 , -0.7309139 , -0.52631956,
        -0.6803367 , -0.9744914 , -0.9639372 ,  0.8591712 , -0.07826253,
         0.9617582 , -0.7269689 , -0.9957466 , -0.6216118 , -0.9720878 ,
         0.89779437,  0.93081397, -0.9966131 ,  0.9854572 ,  0.9793006 ]],
      dtype=float32), array([[-3.2502072 , -4.0018077 , -2.370447  ,  3.8773093 ,  3.6909149 ,
        -3.2669096 ,  3.1789834 , -3.9679527 ,  3.662548  ,  4.506039  ,
        -2.5889246 , -1.4275558 , -2.43